In [25]:
import xml.etree.ElementTree as ETree
import pandas as pd
import requests
import json  


In [26]:
xmldata = r"C:\Users\16476\Desktop\Itamar's Folder\BCIA Class Notes\RSC885 Capstone Project\MsbRegistryPublicDataFile.xml"
prstree = ETree.parse(xmldata)
root = prstree.getroot()



In [28]:
data = {
    "Status": [],
    "RegistrationDate": [],
    "ExpirationDate": [],
    "CessationDate": [],
    "Activities": [],
    "Name": [],
    "Location": [],
    "Latitude": [],
    "Longitude": []
}

API_KEY = 'NotForYou'
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'

def getCoordinates(msbLocation):
    params = {
        'key': API_KEY,
        'address': msbLocation
    }
    response = requests.get(base_url, params=params).json()
    if response['status'] == 'OK':
        return response['results'][0]['geometry']['location']
    return { "lat": 0, "lng": 0 }
    

def buildAddress(msbLocationInformation):
    msbLocation = "" #default location
    # Check if location exists
    if msbLocationInformation is not None:
        # Build address
        msbLocationParent =  msbLocationInformation.find('MsbLocation')
        street = ""
        city = ""
        province = ""
        zipcode = ""
        if msbLocationParent.find('StreetAddress') is not None:
            street = msbLocationParent.find('StreetAddress').text
        if msbLocationParent.find('CityName') is not None:
            city = msbLocationParent.find('CityName').text
        if msbLocationParent.find('AlphaProvinceCode') is not None:
            province = msbLocationParent.find('AlphaProvinceCode').text
        if msbLocationParent.find('PostalZipCode') is not None:
            zipcode = msbLocationParent.find('PostalZipCode').text
        msbLocation = street + ", " + city + ", " + province + ", " + zipcode
    return msbLocation

def addActivities(msbinfo):
    msbActivities = ""
    for msbInfoActivities in msbinfo.iter("MsbBusinessActivity"):
        activity = msbInfoActivities.find('ActivityDescriptionEnglish').text
        if len(msbActivities) > 0:
             msbActivities = msbActivities + ", " + activity
        else:
            msbActivities = activity
    return msbActivities
    
for msbinfo in root.iter('MsbInformation'):
    msbStatus = msbinfo.find('MsbRegistrationStatus').find('StatusDescriptionEnglish').text
    msbRegistrationDate = msbinfo.find('MsbRegistrationDate').text
    msbExpirationDate = ""
    if msbinfo.find('MsbRegistrationExpirationDate') is not None:
        msbExpirationDate = msbinfo.find('MsbRegistrationExpirationDate').text
    msbCessationDate = ""
    if msbinfo.find('MsbCessationDate') is not None:
        msbCessationDate = msbinfo.find('MsbCessationDate').text
    msbName = ""
    if msbinfo.find('MsbLegalBusinessName') is not None:
        msbName = msbinfo.find('MsbLegalBusinessName').find('Name').text
    
    # Location
    msbLocationInformation = msbinfo.find('MsbLocationInformation')
    msbLocation = buildAddress(msbLocationInformation)
    
    coordinates = getCoordinates(msbLocation)
    lat = coordinates['lat']
    long = coordinates['lng']
    
    
    # Activities
    msbActivities = addActivities(msbinfo)
    
    data["Status"].append(msbStatus)
    data["RegistrationDate"].append(msbRegistrationDate)
    data["ExpirationDate"].append(msbExpirationDate)
    data["CessationDate"].append(msbCessationDate)
    data["Activities"].append(msbActivities)
    data["Location"].append(msbLocation)
    data["Name"].append(msbName)
    data["Latitude"].append(lat)
    data["Longitude"].append(long)
    
df = pd.DataFrame(data)
df
    

,Status,RegistrationDate,ExpirationDate,CessationDate,Activities,Name,Location,Latitude,Longitude
0,Ceased,2012-01-18,,2013-05-01,"Foreign exchange dealing, Money transferring, ...",INTERNATIONAL FINANCIAL EMPIRE LTD,"3933 14 IÈME RUE, LAVAL, QC, H7R2S1",45.554503,-73.874506
1,Expired,2012-03-09,2015-01-31,,"Foreign exchange dealing, Money transferring",BUREAU DE CHANGE PRIMEX INC.,"1045 ST-VIATEUR AVE, MONTREAL, QC, H2V1Y4",45.521124,-73.603597
2,Ceased,2012-02-15,,2016-08-30,Foreign exchange dealing,,"2419 1 AVE NW, CALGARY, AB, T2N 0C1",51.053247,-114.116593
3,Ceased,2012-01-18,,2018-08-31,"Foreign exchange dealing, Money transferring",NCRYPT FINANCIAL CORP.,"915-1045 HOWE STREET, VANCOUVER, BC, V6Z2A9",49.280784,-123.122692
4,Expired,2012-03-14,2015-01-31,,"Foreign exchange dealing, Money transferring",YAVADE LTD,"14-30 EGLINTON AVE WEST UNIT 111, MISSISSSAUGA...",43.605496,-79.653029
...,...,...,...,...,...,...,...,...,...
4143,Registered,2023-01-12,2025-01-13,,"Money transferring, Payment service provider",MYBANC INC,"402- 4080 CONFEDERATION PARKWAY, MISSISSAUGA, ...",43.593187,-79.655735
4144,Registered,2023-01-19,2025-06-30,,Dealing in virtual currencies,TRUBIT LTD,"6141 PRESERVE DR, PLANO, TX, 75024",33.084340,-96.829049
4145,Registered,2023-01-19,2025-01-20,,"Money transferring, Payment service provider",FLEETOPERATE INC.,"2 ABBOTSFIELD DRIVE, WINNIPEG, MB, R2N3Y2",49.814532,-97.086213
4146,Registered,2023-01-12,2025-01-31,,Dealing in virtual currencies,PIASTRA PAY INC.,"810 QUAYSIDE DRIVE, SUITE 205, NEW WESTMINSTER...",49.200015,-122.911731


In [29]:
df.to_csv(r"C:\Users\16476\Desktop\Itamar's Folder\BCIA Class Notes\RSC885 Capstone Project\20230216_MSBParsedData_IZ.csv", encoding='utf-8', index=False)